# Assignment 9: Logistic Regression

## Dataset: Titanic Survival Prediction

**Objective:** Predict passenger survival using Logistic Regression.

**Topics Covered:**
- Logistic Regression
- Binary Classification
- ROC Curve and AUC
- Model Interpretation

---
## Step 1: Import Libraries and Load Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Load the dataset
df_train = pd.read_csv('Titanic_train.csv')
df_test = pd.read_csv('Titanic_test.csv')

print("Training set loaded! Shape:", df_train.shape)
print("Testing set loaded! Shape:", df_test.shape)
print("\nFirst 5 rows of training data:")
df_train.head()

---
## Step 2: Exploratory Data Analysis (EDA)

In [ ]:
# Check data types and summary
print("=== Data Types ===")
print(df_train.dtypes)

print("\n=== Summary Statistics ===")
df_train.describe()

In [ ]:
# Check missing values
print("=== Missing Values ===")
missing = df_train.isnull().sum()
missing_pct = (missing / len(df_train)) * 100

for col in df_train.columns:
    if missing[col] > 0:
        print(col + ":", missing[col], "(", round(missing_pct[col], 2), "%)")

In [ ]:
# Target variable distribution
print("=== Survival Distribution ===")
survival_counts = df_train['Survived'].value_counts()
print(survival_counts)

plt.figure(figsize=(8, 5))
plt.bar(['Not Survived', 'Survived'], survival_counts.values, color=['red', 'green'])
plt.xlabel('Survival Status')
plt.ylabel('Count')
plt.title('Survival Distribution')

for i in range(len(survival_counts)):
    plt.text(i, survival_counts.values[i] + 10, str(survival_counts.values[i]), ha='center')

plt.savefig('survival_distribution.png')
plt.show()

In [ ]:
# Survival by Sex
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
sex_survival = df_train.groupby('Sex')['Survived'].mean()
plt.bar(sex_survival.index, sex_survival.values, color=['blue', 'pink'])
plt.xlabel('Sex')
plt.ylabel('Survival Rate')
plt.title('Survival Rate by Sex')

# Survival by Pclass
plt.subplot(1, 2, 2)
class_survival = df_train.groupby('Pclass')['Survived'].mean()
plt.bar(['1st', '2nd', '3rd'], class_survival.values, color=['gold', 'silver', 'brown'])
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate')
plt.title('Survival Rate by Class')

plt.tight_layout()
plt.savefig('survival_analysis.png')
plt.show()

In [ ]:
# Age distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_train['Age'].dropna(), bins=30, color='steelblue', edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution')

plt.subplot(1, 2, 2)
plt.hist(df_train['Fare'], bins=30, color='green', edgecolor='black')
plt.xlabel('Fare')
plt.ylabel('Frequency')
plt.title('Fare Distribution')

plt.tight_layout()
plt.savefig('distributions.png')
plt.show()

---
## Step 3: Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df = df_train.copy()

# Handle missing values
print("=== Handling Missing Values ===")

# Fill Age with median
age_median = df['Age'].median()
df['Age'] = df['Age'].fillna(age_median)
print("Filled Age with median:", age_median)

# Fill Embarked with mode
embarked_mode = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(embarked_mode)
print("Filled Embarked with mode:", embarked_mode)

# Drop Cabin (too many missing values)
df = df.drop('Cabin', axis=1)
print("Dropped Cabin column")

print("\nRemaining missing values:", df.isnull().sum().sum())

In [ ]:
# Encode categorical variables
print("=== Encoding Categorical Variables ===")

# Encode Sex (Male=1, Female=0)
df['Sex_encoded'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
print("Sex: male=1, female=0")

# Encode Embarked
embarked_mapping = {'S': 0, 'C': 1, 'Q': 2}
df['Embarked_encoded'] = df['Embarked'].map(embarked_mapping)
print("Embarked: S=0, C=1, Q=2")

df.head()

In [ ]:
# Select features for model
feature_cols = ['Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_encoded']

X = df[feature_cols]
y = df['Survived']

print("Features:", feature_cols)
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", len(X_train))
print("Testing set:", len(X_test))

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")

---
## Step 4: Model Building

In [ ]:
# Build Logistic Regression model
print("=== Training Logistic Regression Model ===")

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")

In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

print("Predictions made on test set")

---
## Step 5: Model Evaluation

In [ ]:
# Calculate metrics
print("=== Model Evaluation ===")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1-Score:", round(f1, 4))
print("ROC-AUC:", round(roc_auc, 4))

In [ ]:
# Confusion Matrix
print("=== Confusion Matrix ===")

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Survived', 'Survived'],
            yticklabels=['Not Survived', 'Survived'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

print("\nTrue Negatives:", cm[0][0])
print("False Positives:", cm[0][1])
print("False Negatives:", cm[1][0])
print("True Positives:", cm[1][1])

In [ ]:
# ROC Curve
print("=== ROC Curve ===")

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label='ROC Curve (AUC = ' + str(round(roc_auc, 4)) + ')')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.savefig('roc_curve.png')
plt.show()

In [ ]:
# Classification Report
print("=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=['Not Survived', 'Survived']))

---
## Step 6: Model Interpretation

In [ ]:
# Interpret coefficients
print("=== Logistic Regression Coefficients ===")

coefficients = model.coef_[0]
intercept = model.intercept_[0]

print("\nIntercept:", round(intercept, 4))
print("\nCoefficients:")
for i in range(len(feature_cols)):
    coef = coefficients[i]
    feature = feature_cols[i]
    print("  " + feature + ":", round(coef, 4))

In [ ]:
# Feature Importance (based on absolute coefficient value)
print("=== Feature Importance ===")

importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
})

importance_df = importance_df.sort_values('Abs_Coefficient', ascending=True)

# Plot
plt.figure(figsize=(10, 6))
colors = ['green' if c > 0 else 'red' for c in importance_df['Coefficient']]
plt.barh(importance_df['Feature'], importance_df['Coefficient'], color=colors)
plt.xlabel('Coefficient')
plt.ylabel('Feature')
plt.title('Logistic Regression Coefficients')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.savefig('feature_importance.png')
plt.show()

print("\nInterpretation:")
print("- Positive coefficient = increases survival probability")
print("- Negative coefficient = decreases survival probability")
print("- Sex_encoded (negative): Being male reduces survival chances")
print("- Pclass (negative): Higher class number (3rd class) reduces survival")

---
## Step 7: Save Model for Deployment

In [ ]:
# Save model for Streamlit deployment
import pickle

# Save model
with open('logistic_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model and scaler saved for deployment!")

---
## Interview Questions

### 1. Difference between Precision and Recall

| Metric | Formula | Meaning |
|--------|---------|--------|
| **Precision** | TP / (TP + FP) | Of all predicted positives, how many are actually positive? |
| **Recall** | TP / (TP + FN) | Of all actual positives, how many did we correctly predict? |

**When to use which:**
- **High Precision needed:** When false positives are costly (e.g., spam detection)
- **High Recall needed:** When false negatives are costly (e.g., disease detection)

### 2. Cross-Validation in Binary Classification

**What is Cross-Validation?**
- Technique to evaluate model by splitting data into multiple folds
- Train on some folds, test on remaining fold
- Repeat for all folds and average results

**Why is it important?**
1. Gives more reliable estimate of model performance
2. Reduces overfitting by testing on different data subsets
3. Uses all data for both training and testing
4. Helps in hyperparameter tuning

---
## Summary

In this assignment, we:

1. **Explored** the Titanic dataset and visualized survival patterns
2. **Preprocessed** data by handling missing values and encoding categoricals
3. **Built** a Logistic Regression model
4. **Evaluated** using accuracy, precision, recall, F1, and ROC-AUC
5. **Interpreted** coefficients to understand feature importance
6. **Saved** model for Streamlit deployment